# Protein folding with Metropolis and Quantum Walks

Work plan

1. Program Metropolis in superposition using Qiskit. From arXiv:1910.01659
3. Figure out how to rotate the structure of the proteins. Perhaps use psi4 to figure out the energy of each configuration
4. Prove it with usual Metropolis and Tripeptides: https://en.wikipedia.org/wiki/Tripeptide

Since we are going to use IBM simulator, we can use up to 32 qubits!

Remember that to convert some subcircuit to a gate, one uses:
sub_inst = sub_circ.to_instruction()
circ.append(sub_inst, [q[1], q[2]])

In [2]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.12.0',
 'qiskit-aer': '0.4.0',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.4.6',
 'qiskit-aqua': '0.6.4',
 'qiskit': '0.15.0'}

In [8]:
# Import libraries
from qiskit import *
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit, Qubit, Clbit, Gate 
from qiskit.aqua.components.oracles import Oracle, TruthTableOracle
import numpy as np
import math

import logging

from sympy.combinatorics.graycode import GrayCode
from math import pi

from qiskit.aqua.utils.controlled_circuit import apply_cu3
from qiskit.aqua import AquaError

%matplotlib inline

The first thing we need to do is to calculate the registers we need in order to be able to perform the metropolis algorithm:

For example, for a tripeptide (n=3) we have 2(n-1) angles

Coin: 
1 to allow or decline move

Move: 2 moves on 4 angles:
2 to identify angle
1 to identify if we increase or decrease it

State: 32 divisions (5 qubits) for each angle
20 qubits = 4 angles * 5 qubits/angle

Ancillas: 
8 qubits

Total: 32 qubits, maximum

In [4]:
#Indicate number of precision bits
precision_bits = 5

ancilla_bits = 10

In [3]:
# For tripeptide
'''
# State definition. 32 divisions for each of the four angles. All angles range from 0 to 2pi
angle_0 = QuantumRegister(5, name = 'angle_0')
angle_1 = QuantumRegister(5, name = 'angle_1') 
angle_2 = QuantumRegister(5, name = 'angle_2') 
angle_3 = QuantumRegister(5, name = 'angle_3') 


# Move proposal
move_id = QuantumRegister(2, name = 'move_id') #Which angle are we modifying
move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

# Coin
coin = QuantumRegister(1, name = 'coin')

# Ancillas
ancilla = QuantumRegister(8, name = 'ancilla')



# Circuit
qc = QuantumCircuit(angle_0,angle_1,angle_2,angle_3,move_id,move_value,coin,ancilla)
'''

# For dipeptide
# State definition. 32 divisions for each of the four angles. All angles range from 0 to 2pi
angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 

# Move proposal
move_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying
move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

# Coin
coin = QuantumRegister(1, name = 'coin')

# Ancillas
ancilla = QuantumRegister(ancilla_bits, name = 'ancilla')


# Circuit
qc = QuantumCircuit(angle_phi,angle_psi,move_id,move_value,coin,ancilla)

Next we have to define the steps of the Metropolis using Quantum Walk

In [4]:
# This is the move preparation gate
def move_preparation(circuit,move_id,move_value):
    '''
    Proposes new moves
    '''
    circuit.h(move_id)
    circuit.h(move_value)

# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
sub_circ = QuantumCircuit(s_move_id, s_move_value)
move_preparation(sub_circ,s_move_id,s_move_value)
move_preparation = sub_circ.to_instruction()


# Use as qc.append(move_preparation, [move_id[0], move_id[1], move_value[0]])

                                 
   angle_0_0: |0>────────────────
                                 
   angle_0_1: |0>────────────────
                                 
   angle_0_2: |0>────────────────
                                 
   angle_0_3: |0>────────────────
                                 
   angle_0_4: |0>────────────────
                                 
   angle_1_0: |0>────────────────
                                 
   angle_1_1: |0>────────────────
                                 
   angle_1_2: |0>────────────────
                                 
   angle_1_3: |0>────────────────
                                 
   angle_1_4: |0>────────────────
                                 
   angle_2_0: |0>────────────────
                                 
   angle_2_1: |0>────────────────
                                 
   angle_2_2: |0>────────────────
                                 
   angle_2_3: |0>────────────────
                                 
   angle_2_4: 

Next, we define the conditional move. For that we have to go over all possible combinations of move_id and move_value, adding or substracting 1, conditioned on the coin being at state 1

In [ ]:
def conditional_move_tripeptide(circuit,coin,move_id,move_value,angle_id,angle_value,ancilla):
    '''
    Conditioned on coin, perform a move. For a tripeptide!
    We use a repetitive structure where we perform the conditional sum and substraction for each angle.
    '''
    # For angle_0 ----------------------------------------------
    circuit.x(move_id) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])
    #circuit.append(Gate("mcx", 5, []), [move_id[0], move_id[1], move_value, coin, ancilla[0]]) #create a single control
    sum1(circuit,angle_0,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_0,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id)
    
    # For angle_1 ----------------------------------------------
    circuit.x(move_id[1]) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_1,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_1,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id[1])
    
    # For angle_2 ----------------------------------------------
    circuit.x(move_id[0]) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_2,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_2,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id[0])
    
    # For angle_3 ----------------------------------------------
    #circuit.x(move_id[0]) #  move_id  is in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_3,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_3,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) = 6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    #circuit.x(move_id[0])
    
# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)
s_ancilla = QuantumRegister(ancilla.size)
s_angle_0 = QuantumRegister(5, name = 'angle_0')
s_angle_1 = QuantumRegister(5, name = 'angle_1') 
s_angle_2 = QuantumRegister(5, name = 'angle_2') 
s_angle_3 = QuantumRegister(5, name = 'angle_3') 

sub_circ = QuantumCircuit(s_angle_0,s_angle_1,s_angle_2,s_angle_3,s_move_id,s_move_value,s_coin,s_ancilla)
conditional_move_tripeptide(sub_circ,s_coin,s_move_id,s_move_value,s_angle_id,s_angle_value,s_ancilla)
conditional_move = sub_circ.to_instruction()

qc.append(move_preparation.inverse(), [(s_angle_0[j] for j in range(4)),(s_angle_1[j] for j in range(4)),(s_angle_2[j] for j in range(4)),(s_angle_3[j] for j in range(4)),move_id[0], move_id[1], move_value[0]])

In [7]:
def conditional_move_dipeptide(circuit,coin,move_id,move_value,angle_id,angle_value,ancilla):
    '''
    Conditioned on coin, perform a move. For a dipeptide!
    We use a repetitive structure where we perform the conditional sum and substraction for each angle.
    '''
    # For angle_phi = angle_id = 0 ----------------------------------------------
    circuit.x(move_id) # Put move_id in 1
    
    # Conditional on move_id = 0, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])
    #circuit.append(Gate("mcx", 5, []), [move_id[0], move_id[1], move_value, coin, ancilla[0]]) #create a single control
    sum1(circuit,angle_phi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_phi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id)
    
    # For angle_psi = angle_id = 1 ----------------------------------------------
    # Put move_id in 11
    
    # Conditional on move_id = 1, move_value = 1 and coin = 1, increase angle_psi by one
    circuit.mcrx(theta = pi, q_controls = [move_id[0],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_psi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 1, move_value = 0 and coin = 1, decrease angle_psi by one
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_psi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    
# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)
s_ancilla = QuantumRegister(ancilla.size)
s_angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
s_angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 


sub_circ = QuantumCircuit(s_angle_phi,s_angle_psi,s_move_id,s_move_value,s_coin,s_ancilla)
conditional_move_dipeptide(sub_circ,s_coin,s_move_id,s_move_value,s_angle_id,s_angle_value,s_ancilla)
conditional_move = sub_circ.to_instruction()

# To use it or its inverse
# qc.append(move_preparation.inverse(), [(s_angle_phi[j] for j in range(precision_bits)),(s_angle_psi[j] for j in range(precision_bits)),move_id[0], move_value[0]])

In [6]:
def reflection(circuit,move_id,move_value,coin):
    '''
    I have to investigate over what is the reflection performed. Is it performed over 000?
    If in state 0000, make it 1111, cccz gate and back to 0000
    '''
    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)
    
    circuit.mcrz(theta = pi, q_controls = move_id + [move_value], q_target = coin)
    
    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)
    
# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)



sub_circ = QuantumCircuit(s_move_id,s_move_value,s_coin)
reflection(sub_circ,s_move_id,s_move_value,s_coin)
reflection = sub_circ.to_instruction()

# Use as qc.append(reflection.inverse(), [move_id[0], move_value[0],coin[0]])

In [74]:
def coin_flip(circuit,coin,ancilla,direct = 1):
    '''
    Prepares the coin with the probability encoded in the ancilla, or unprepares it if direct == -1
    direct: if coin_flip then direct = 1
            if inverse_coin_flip direct = -1
    '''
    
    #Necesitamos usar el número guardado en las ancillas para realizar rotaciones controladas.  
    #Notice that ancilla encodes 1-probability, rather than probability.
    #Notice also that cu3(theta) rotates theta/2. As the first angle to rotate is pi/4 we need to start in theta = pi/2
    
    if direct == 1:
        circuit.x(coin)
        for i in range(ancilla.size):
            circuit.cu3(theta = -math.pi/(2**(i+1)), phi  = 0, lam = 0, control_qubit = ancilla[i], target_qubit = coin)

    else:
        for i in range(ancilla.size-1,-1,-1):
            circuit.cu3(theta = math.pi/(2**(i+1)), phi  = 0, lam = 0, control_qubit = ancilla[i], target_qubit = coin)
        circuit.x(coin)

We now define the coin flip, which takes three steps: 
1. Calculating the oracle that outputs the probability encoded in the ancilla
2. Performing the coin flip
3. Undoing the oracle to uncompute the ancilla

We create a subcircuit sub_circ with registers of type 's_ ' where we perform all the calculations.
Unfortunately, we will have to go over all this next lines in order to perform the entire calculation, which cannot be packed in a gate due to the dependence to the beta parameter.

Use as main_circuit += sub_circ

In [103]:
#beta = Parameter('beta')
oracle = beta_precalc_TruthTableOracle(energies_dictionary,beta,out_bits = ancilla_bits)

#oracle_gate = oracle_circuit.to_instruction()


# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)
s_ancilla = QuantumRegister(ancilla.size)
s_angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
s_angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 



[s_angle_phi, s_angle_psi, s_move_id, s_move_value] = oracle.variable_register 
s_ancilla = oracle.output_register 
sub_circ = QuantumCircuit(oracle.variable_register,s_coin,s_ancilla)


#print(type(sub_circ))


oracle.construct_circuit()
oracle_circuit = oracle.circuit
oracle_gate = oracle_circuit.to_instruction()

sub_circ += oracle_circuit
coin_flip(sub_circ,s_coin,s_ancilla)
sub_circ.append(oracle_gate.inverse(), [oracle.variable_register, s_ancilla])

# Use as main_circuit += sub_circ
'''
To apply the inverse
sub_circ += oracle_circuit
coin_flip(sub_circ,s_coin,s_ancilla, direct = -1)
sub_circ.append(oracle_gate.inverse(), [oracle.variable_register, s_ancilla])
'''

NameError: name 'energies_dictionary' is not defined

Next thing we should do is create the entire loop.

# Helper functions

In [8]:
def sum1(circuit,qubit_string,control,start,end):
    '''
    Sums register 1 and 2 (1 qubit) in register 1. Tested ok.
    '''
    n_qubits = qubit_string.size     # calculate n_qubits
    circuit.cx(control,end) # iff control = 1, end = 1
    circuit.x(start)
    circuit.cx(control,start) # iff control = 1, start = 0
    
    for i in range(n_qubits+1): #Don't need to add control, since start already does that work
        '''
        Next thing we analise if all qubits to the right have value 1, 
        and save it in the current qubit and start
        '''
        if i > 0:
            # For i = 0, there is only the start to worry about
            circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end[0]], q_target = qubit_string[n_qubits-i])
        circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end[0]], q_target = start[0])

        '''
        Next, controlling on the current qubit and start, we change all the following qubits to 0.
        We have to control with qubit_string[n_qubit]
        '''
        if i == 0:
            for j in range(n_qubits-i):
                circuit.ccx(control,start,qubit_string[j])
            circuit.ccx(control,start,end)
        elif i == n_qubits:
            circuit.mcrx(theta = pi, q_controls = [control[0],qubit_string[n_qubits-i],start[0]], q_target = end[0])
        else:
            for j in range(n_qubits-i):            
                circuit.mcrx(theta = pi, q_controls = [control[0],qubit_string[n_qubits-i],start[0]], q_target = qubit_string[j])
            circuit.mcrx(theta = pi, q_controls = [control[0],qubit_string[n_qubits-i],start[0]], q_target = end[0])
    circuit.x(start)

In [9]:
def substract1(circuit,qubit_string,control,start,end):
    '''
    In binary, substracting is the same procedure as summing when we exchange 0s and 1s
    '''
    circuit.x(qubit_string)

    sum1(circuit,qubit_string,control,start,end)
    
    circuit.x(qubit_string)

In [63]:
class beta_precalc_TruthTableOracle(TruthTableOracle):
    '''Outputs the binary angle of rotation to get the correct probability'''
    def __init__(self, energies_dictionary, beta, out_bits, optimization=False, mct_mode='basic'):
        self.beta = beta
        self.out_bits = out_bits
        self.in_bits = len(list(energies_dictionary.keys())[0]) # The key of energies_dictionary is the input bits
        self.energies_dictionary = energies_dictionary
        self.calculate_bitmap()
        super().__init__(self.bitmap, optimization, mct_mode)
        
    def calculate_bitmap(self):
        new_bitmap = []
        angles = {}
        for i in range(int(2**self.in_bits)):
            if self.energies_dictionary[str(i)] >= 0:
                probability = np.exp(-self.beta * self.energies_dictionary[str(i)])
                print('probability: ',probability)
            else: 
                probability = 1
                
            # Instead of encoding the probability, we will encode 1-probability. That way 1 -> 000, 
            #but if probability is 0 there is some small probability of acceptance
            probability = 1 - probability
            
            
            # Instead of probability save angles so rotations are easier to perform afterwards sqrt(p) = sin(theta)
            angle = np.arcsin(np.sqrt(probability)) 
            
            # Make the angle be between [0,1]. Since the maximum is pi/2
            angle /= (np.pi/2) 
            
            # Convert it into an integer and a string
            int_angle = format(int(angle*2**self.out_bits), 'b')
            str_angle = str(int_angle) 
            
            # Convert it to binary
            int_angle = format(int(angle*2**out_bits), 'b')
            if int_angle == '1' + '0'*out_bits:
                angles[str(i)] = '1'*out_bits # As we only have out_bits, the 10000 is substituted by 1111
            else:
                str_angle = str(int_angle)
                angles[str(i)] = '0'*(out_bits - len(str_angle)) + str_angle
            
             
        #print(new_probabilities)
        # Encoding the new bitmap
        new_bitmap = []
        for o in range(self.out_bits):
            string = ''
            for i in range(int(2**self.in_bits)):
                string += str(angles[str(i)])[o]
            new_bitmap += [string]
        #print(new_bitmap)
        self.bitmap = new_bitmap

# Experiments without relevance

In [5]:
n_qubits = 3
qubit_string = QuantumRegister(n_qubits,'string')
print(type(qubit_string))
control = QuantumRegister(1,'control')
start = QuantumRegister(1,'start')
end = QuantumRegister(1,'end')

cqubit_string = ClassicalRegister(n_qubits,'cstring')
ccontrol = ClassicalRegister(1,'ccontrol')
cstart = ClassicalRegister(1,'cstart')
cend = ClassicalRegister(1,'cend')

circuit = QuantumCircuit(qubit_string,control,start,end,cqubit_string,ccontrol,cstart,cend)

#Choose state
circuit.x(control)

#Sum1
sum1(circuit,qubit_string,control,start,end)

#Measure
circuit.measure(qubit_string,cqubit_string)
circuit.measure(control,ccontrol)
circuit.measure(start,cstart)
circuit.measure(end,cend)


# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(circuit, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(circuit)

counts

<class 'qiskit.circuit.quantumregister.QuantumRegister'>


AttributeError: 'QuantumCircuit' object has no attribute 'mcrx'

In [98]:
ancilla = QuantumRegister(6,'a')
coin = QuantumRegister(1,'c')

c_ancilla = ClassicalRegister(6,'ca')
c_coin = ClassicalRegister(1,'cc')

circ = QuantumCircuit(ancilla,coin,c_ancilla,c_coin)

#--- Preparación de la ancilla

circ.x(ancilla[1])
circ.x(ancilla[3])
circ.x(ancilla[5])

#--- 

circ.barrier(ancilla,coin)
coin_flip(circ,coin,ancilla)
circ.barrier(ancilla,coin)
circ.measure(ancilla,c_ancilla)
circ.measure(coin,c_coin)

print(circ)


              ░                                                         »
a_0: |0>──────░──────────────■──────────────────────────────────────────»
        ┌───┐ ░              │                                          »
a_1: |0>┤ X ├─░──────────────┼────────────────■─────────────────────────»
        └───┘ ░              │                │                         »
a_2: |0>──────░──────────────┼────────────────┼────────────────■────────»
        ┌───┐ ░              │                │                │        »
a_3: |0>┤ X ├─░──────────────┼────────────────┼────────────────┼────────»
        └───┘ ░              │                │                │        »
a_4: |0>──────░──────────────┼────────────────┼────────────────┼────────»
        ┌───┐ ░              │                │                │        »
a_5: |0>┤ X ├─░──────────────┼────────────────┼────────────────┼────────»
        └───┘ ░ ┌───┐┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐»
c_0: |0>──────░─┤ X ├┤ U3(-pi/2,0,0) ├

In [102]:
# Import Aer
from qiskit import Aer

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(circ, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(circ)
print(counts)


{'0 101010': 264, '1 101010': 760}


In [97]:
probability = 0.25
out_bits = 6

# Instead of encoding the probability, we will encode 1-probability. That way 1 -> 000, 
#but if probability is 0 there is some small probability of acceptance
#probability = 1 - probability


# Instead of probability save angles so rotations are easier to perform afterwards
angle = np.arcsin(np.sqrt(probability))

print('original angle: ',angle)
print((np.sin(angle))**2)

# Make the angle be between [0,1]. Since the maximum is pi/2
angle /= (np.pi/2) 
#print(np.pi/2)
print('formated angle',angle)

# The self.out_bits-1 appears because we reserve the 10000 for probability = 1
int_angle = format(int(angle*2**out_bits), 'b')
if int_angle == '1' + '0'*out_bits:
    str_angle = '1'*out_bits
else:
    str_angle = str(int_angle)
print('angle recorded: ',int(str_angle,2),'/',2**out_bits)

print(str_angle)
print('0'*(out_bits - len(str_angle)) + str_angle)
print((np.sin(int(str_angle,2)/2**(out_bits)*np.pi/2))**2)

original angle:  0.5235987755982988
0.24999999999999994
formated angle 0.3333333333333333
angle recorded:  21 / 64
10101
010101
21
0.24294862790338914


In [68]:
out_bits = 4
probability = 3/4
angle = np.arcsin(np.sqrt(probability))
angle /= (np.pi/2)
angle = format(int(angle*2**out_bits), 'b')
print(angle)

1010
